In [15]:
# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql
import hashlib
import pyprind
import numpy as np

# We are also going to do some basic viz
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
connection_args = {
    'host': 'localhost', # You'll have to update this to your IP
    'user': 'postgres',  # username from above
    'password' : 'sharkbolo88$$', #password
    'dbname': 'goodreads',   # DB that we are connecting to
    'port': 5432         # port we opened on AWS
}

# We will talk about this magic Python trick!
connection = pg.connect(**connection_args)

In [3]:
query = "SELECT * FROM scrapes LIMIT 10;"

pd_sql.read_sql(query, connection)

,userid,shelf,review_url,title,author,avg_rating,times_rated,rating,comments,likes,date_added,isbn,bookid
0,11372510,favorites,/review/show/2471723375,"Saint's Blood (Greatcoats, #3)","de Castell, Sebastien",4.30,4417,5,0,1,"Jul 26, 2018",1782066802,"de Castell, Sebastien || Saint's Blood (Greatc..."
1,11372510,favorites,/review/show/2471723184,"Knight's Shadow (Greatcoats, #2)","de Castell, Sebastien",4.26,6630,5,0,3,"Jul 26, 2018",1782066772,"de Castell, Sebastien || Knight's Shadow (Grea..."
2,11372510,favorites,/review/show/2439946945,"Traitor's Blade (Greatcoats, #1)","de Castell, Sebastien",4.07,12790,5,0,0,"Jun 29, 2018",184866379X,"de Castell, Sebastien || Traitor's Blade (Grea..."
3,11372510,favorites,/review/show/2378103609,You Are a Badass: How to Stop Doubting Your Gr...,"Sincero, Jen",4.00,73220,5,0,0,"Apr 30, 2018",0762447699,"Sincero, Jen || You Are a Badass: How to Stop ..."
4,11372510,favorites,/review/show/2175881035,"Spectyr (Book of the Order, #2)","Ballantine, Philippa",3.88,1546,4,0,0,"Nov 06, 2017",0441020518,"Ballantine, Philippa || Spectyr (Book of the O..."
5,11372510,favorites,/review/show/2099803698,Lily and the Octopus,"Rowley, Steven",3.72,24260,5,0,0,"Aug 21, 2017",1501126229,"Rowley, Steven || Lily and the Octopus"
6,11372510,favorites,/review/show/1757025829,"Warrior Witch (The Malediction Trilogy, #3)","Jensen, Danielle L.",3.89,3963,5,0,0,"Sep 13, 2016",0857664697,"Jensen, Danielle L. || Warrior Witch (The Male..."
7,11372510,favorites,/review/show/1757025560,"Hidden Huntress (The Malediction Trilogy, #2)","Jensen, Danielle L.",3.88,7850,5,0,0,"Sep 13, 2016",0857664727,"Jensen, Danielle L. || Hidden Huntress (The Ma..."
8,11372510,favorites,/review/show/1751040809,"Tower of Dawn (Throne of Glass, #6)","Maas, Sarah J.",4.44,57102,5,0,0,"Sep 07, 2016",1681195771,"Maas, Sarah J. || Tower of Dawn (Throne of Gla..."
9,11372510,favorites,/review/show/1747635071,"Geist (Book of the Order, #1)","Ballantine, Philippa",3.64,3642,5,0,0,"Sep 03, 2016",0441019617,"Ballantine, Philippa || Geist (Book of the Ord..."


In [4]:
query = "SELECT userid, bookid, isbn, rating, avg_rating FROM scrapes;"

df = pd_sql.read_sql(query, connection)

In [5]:
print(df.shape)
df = df[df.rating <= 5]
df = df[df.rating >= 1]
print(df.shape)

(11448402, 5)
(8382766, 5)


In [7]:
def isbn_hash(row):
    isbn= row['isbn']
    bookid =row['bookid']
    if isbn == 'no_isbn':
        isbn = 'hash_'+ hashlib.md5(bookid.encode('utf-8')).hexdigest()
    else:
        isbn = 'isbn_'+str(isbn)
    return isbn

In [8]:
a = df.apply(isbn_hash, axis=1)

df.isbn = a 

In [9]:
df.drop_duplicates(subset=['userid', 'isbn'], inplace=True)
df.shape

(544768, 5)

In [18]:
df['rating_delta'] = df.rating-df.avg_rating

mu = np.mean(df.rating_delta)
sigma = np.std(df.rating_delta)

print(mu, sigma)

df['rating_delta_std'] = (df.rating_delta-mu)/sigma

-0.22750903136749834 0.9531088082136135


In [19]:
df.to_csv('10-22-hashed.csv', index=False)

In [21]:
df.sample(10)

,userid,bookid,isbn,rating,avg_rating,rating_delta,rating_delta_std
424549,76842,"Farmer, Jared || Glen Canyon Dammed: Inventing...",isbn_0816519927,3,3.97,-0.97,-0.779020
558131,841652,"Andersen, Sarah || Big Mushy Happy Lump (Sarah...",isbn_1449479618,4,4.31,-0.31,-0.086549
821626,28470999,"Sixx, Nikki || The Heroin Diaries: A Year in t...",isbn_1416511946,5,4.12,0.88,1.161996
876298,8992310,"Roberts, Jane || The Nature of Personal Realit...",isbn_1878424068,4,4.44,-0.44,-0.222945
1166867,166687,"Review, The Paris || Playwrights at Work",isbn_0679640215,5,4.38,0.62,0.889205
808812,25360324,"Raihan, Zahir || কয়েকটি মৃত্যু",isbn_9844040965,3,3.63,-0.63,-0.422293
331854,5315988,"Peacock, Kathleen || Thornhill (Hemlock, #2)",isbn_0062048686,5,3.91,1.09,1.382328
1139438,5024047,"Dorsey, Tim || When Elves Attack: A Joyous Chr...",hash_7870ca83a4957e2b4d606b08347b4438,4,3.80,0.20,0.448542
1042326,111921,"Hemingway, Ernest || The Old Man and the Sea",isbn_0684830493,3,3.75,-0.75,-0.548197
89669,7264121,"Sands, Lynsay || A Quick Bite (Argeneau #1)",isbn_0061961639,3,3.91,-0.91,-0.716068


In [24]:
cursor = connection.cursor()

In [ ]:
#cursor.execute("CREATE TABLE user_book (userid TEXT, PRIMARY KEY (userid));")

#cursor.execute("CREATE TABLE trial (userid TEXT, PRIMARY KEY (userid));")

# trial = df.sample(10)
# trial

# for row in trial.itertuples():
#     print(row[1], row[3], row[4])



# cursor.execute("rollback")

# cursor.execute('INSERT INTO trial (userid, test_book) VALUES (1234, "this is a test")')

# for row in trial.itertuples():
#     userid = str(row[1])
#     isbn = row[3]
#     rating = str(row[4])
#     statement = "ALTER TABLE trial ADD "+isbn+" INT;"
#     print(statement)
#     try:
#         cursor.execute(statement)
#     except:
#         cursor.execute("rollback")
#         pass

# for row in trial.itertuples():
#     userid = str(row[1])
#     isbn = row[3]
#     rating = str(row[4])
#     statement = "INSERT INTO trial " + "(userid, "+isbn + ")VALUES ("+userid+", "+rating + ");"
#     try:
#         cursor.execute(statement)
#     except:
#         cursor.execute("rollback")
#         pass

In [22]:
df.shape

(544768, 7)

In [106]:
cursor.execute("CREATE TABLE user_book (userid TEXT);")

In [107]:
df.isbn.unique()[0:10]

array(['isbn_1782066802', 'isbn_1782066772', 'isbn_184866379X',
       'isbn_0762447699', 'isbn_0441020518', 'isbn_1501126229',
       'isbn_0857664697', 'isbn_0857664727', 'isbn_1681195771',
       'isbn_0441019617'], dtype=object)

In [108]:
progbar =  pyprind.ProgBar(len(df.isbn.unique()))

for isbn in df.isbn.unique():
    statement = "ALTER TABLE user_book ADD "+isbn+" FLOAT;"
    progbar.update()
    try:
        cursor.execute(statement)
    except:
        cursor.execute("rollback")
        pass

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:13


In [27]:
df.head()

,userid,bookid,isbn,rating,avg_rating,rating_delta,rating_delta_std
0,11372510,"de Castell, Sebastien || Saint's Blood (Greatc...",isbn_1782066802,5,4.30,0.70,0.973141
1,11372510,"de Castell, Sebastien || Knight's Shadow (Grea...",isbn_1782066772,5,4.26,0.74,1.015109
2,11372510,"de Castell, Sebastien || Traitor's Blade (Grea...",isbn_184866379X,5,4.07,0.93,1.214456
3,11372510,"Sincero, Jen || You Are a Badass: How to Stop ...",isbn_0762447699,5,4.00,1.00,1.287900
4,11372510,"Ballantine, Philippa || Spectyr (Book of the O...",isbn_0441020518,4,3.88,0.12,0.364606


In [109]:
progbar =  pyprind.ProgBar(len(df.userid.unique()))

for userid in df.userid.unique():
    userid = str(userid)
    statement = "INSERT INTO user_book " + "(userid) VALUES ("+userid+");"
    progbar.update()
    try:
        cursor.execute(statement)
    except:
        cursor.execute("rollback")
        print('error')
        pass

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


In [ ]:
df[df.isbn=='isbn_1937007758']

In [ ]:
    statement = "INSERT INTO user_book " + "(userid, "+isbn + ") VALUES ("+userid+", "+rating + ");"

In [37]:
trial = pd.concat([df.head(5), df.sample(5)])

In [38]:
trial

,userid,bookid,isbn,rating,avg_rating,rating_delta,rating_delta_std
0,11372510,"de Castell, Sebastien || Saint's Blood (Greatc...",isbn_1782066802,5,4.30,0.70,0.973141
1,11372510,"de Castell, Sebastien || Knight's Shadow (Grea...",isbn_1782066772,5,4.26,0.74,1.015109
2,11372510,"de Castell, Sebastien || Traitor's Blade (Grea...",isbn_184866379X,5,4.07,0.93,1.214456
3,11372510,"Sincero, Jen || You Are a Badass: How to Stop ...",isbn_0762447699,5,4.00,1.00,1.287900
4,11372510,"Ballantine, Philippa || Spectyr (Book of the O...",isbn_0441020518,4,3.88,0.12,0.364606
930148,7640495,"Palmer, Diana || The Case Of The Mesmerizing B...",isbn_0373483430,3,3.96,-0.96,-0.768528
1214850,1181579,"Buckley, Carla || The Things That Keep Us Here",isbn_0440245095,5,3.92,1.08,1.371836
642165,37721367,"Tabata, Yuki || ブラッククローバー 2 [Burakku Kurōbā 2]...",isbn_408880452X,3,4.07,-1.07,-0.883940
1268604,3273779,"Murakami, Haruki || The Wind-Up Bird Chronicle",isbn_0965341984,4,4.17,-0.17,0.060338
320357,5081915,"Reid, Stacy || Eternal Darkness (The Amagarian...",hash_f7fa6b734335c26298dfc32fdfd960cd,3,3.97,-0.97,-0.779020


In [116]:
# for userid in df.userid.unique():
#     sub = df[df.userid==userid]
#     index  = list(sub.isbn)
#     values = list(sub.rating_delta_std)
#     pairs = []
#     for a in zip(index, values):
#         pair = str(a[0])+' = '+str(a[1])
#         pairs.append(pair)
#     statement = 'UPDATE user_book SET '+', '.join(pairs)+' WHERE userid = ' + str(userid) +';'
#     try:
#         cursor.execute(statement)
#     except:
#         cursor.execute("rollback")
#         print('error')
#         pass

In [123]:
progbar =  pyprind.ProgBar(len(df))

i =0 

for row in df.itertuples():
    userid = str(row[1])
    isbn = row[3]
    rating_delta_std = str(row[7])
    statement = 'UPDATE user_book SET '+isbn+' = '+rating_delta_std+" WHERE userid = '" + str(userid) +"';"
    print(statement)
    progbar.update()
    cursor.execute(statement)
    i = i +1

UPDATE user_book SET isbn_1782066802 = 0.9731407614476925 WHERE userid = '11372510';
UPDATE user_book SET isbn_1782066772 = 1.0151086875179287 WHERE userid = '11372510';
UPDATE user_book SET isbn_184866379X = 1.2144563363515508 WHERE userid = '11372510';
UPDATE user_book SET isbn_0762447699 = 1.2879002069744647 WHERE userid = '11372510';
UPDATE user_book SET isbn_0441020518 = 0.3646058334292654 WHERE userid = '11372510';
UPDATE user_book SET isbn_1501126229 = 1.581675689466119 WHERE userid = '11372510';
UPDATE user_book SET isbn_0857664697 = 1.4033120036676145 WHERE userid = '11372510';
UPDATE user_book SET isbn_0857664727 = 1.4138039851851738 WHERE userid = '11372510';
UPDATE user_book SET isbn_1681195771 = 0.8262530202018646 WHERE userid = '11372510';
UPDATE user_book SET isbn_0441019617 = 1.6656115416065918 WHERE userid = '11372510';
UPDATE user_book SET hash_322a927ef0e50a92c289341ef957b6c9 = 0.8577289647545422 WHERE userid = '11372510';
UPDATE user_book SET isbn_1501139886 = 0.521

UPDATE user_book SET isbn_076115213X = 0.37509781494682426 WHERE userid = '11372510';
UPDATE user_book SET isbn_0761121323 = 1.5397077633958831 WHERE userid = '11372510';
UPDATE user_book SET hash_58b22f3f5ace309b27d244878c3b01de = 1.130520484211078 WHERE userid = '7133078';
UPDATE user_book SET hash_fe30a282cf9dcf5f809761396d9675c8 = -0.3803248543174296 WHERE userid = '11372510';
UPDATE user_book SET isbn_1250091845 = -0.03408946423797972 WHERE userid = '11372510';
UPDATE user_book SET isbn_0316213101 = -0.653116373773966 WHERE userid = '11372510';
UPDATE user_book SET isbn_0525954538 = 1.5292157818783236 WHERE userid = '11372510';
UPDATE user_book SET isbn_1501107836 = -0.6006564661861703 WHERE userid = '11372510';
UPDATE user_book SET isbn_0765378779 = 0.46952564860485635 WHERE userid = '11372510';
UPDATE user_book SET isbn_0375849726 = -0.6741003368090842 WHERE userid = '11372510';
UPDATE user_book SET isbn_1501141597 = 0.38558979646438357 WHERE userid = '43101170';
UPDATE user_boo

UPDATE user_book SET isbn_0385333846 = 1.2144563363515508 WHERE userid = '7928875';
UPDATE user_book SET isbn_1931520003 = 1.4557719112554104 WHERE userid = '7928875';
UPDATE user_book SET isbn_0156031876 = 1.4662638927729692 WHERE userid = '7928875';
UPDATE user_book SET isbn_1561633194 = 1.1619964287637556 WHERE userid = '7928875';
UPDATE user_book SET isbn_0060515198 = 1.266916243939347 WHERE userid = '7928875';
UPDATE user_book SET isbn_1563890356 = 0.7213332050262742 WHERE userid = '7928875';
UPDATE user_book SET isbn_0590554085 = 1.2879002069744647 WHERE userid = '7928875';
UPDATE user_book SET isbn_0143039830 = 1.3718360591149374 WHERE userid = '7928875';
UPDATE user_book SET isbn_0307347974 = 1.308884170009583 WHERE userid = '7928875';
UPDATE user_book SET isbn_006441034X = 0.9731407614476925 WHERE userid = '7928875';
UPDATE user_book SET isbn_076531178X = 0.8262530202018646 WHERE userid = '7928875';
UPDATE user_book SET isbn_0747263744 = 1.1724884102813145 WHERE userid = '7928

UPDATE user_book SET isbn_0448421658 = -0.9678758193007388 WHERE userid = '187043';
UPDATE user_book SET isbn_0064434516 = -0.9678758193007388 WHERE userid = '187043';
UPDATE user_book SET isbn_006443253X = -1.0727956344763294 WHERE userid = '187043';
UPDATE user_book SET isbn_0020430906 = 0.3436218703941472 WHERE userid = '187043';
UPDATE user_book SET isbn_0060775823 = -3.0243041967423197 WHERE userid = '187043';
UPDATE user_book SET isbn_0440421217 = -1.9541220819512923 WHERE userid = '187043';
UPDATE user_book SET isbn_0208022392 = 0.2701779997712333 WHERE userid = '187043';
UPDATE user_book SET isbn_0802774687 = -0.422292780387666 WHERE userid = '187043';
UPDATE user_book SET isbn_0394818237 = 1.0360926505530466 WHERE userid = '187043';
UPDATE user_book SET isbn_0670557110 = -0.9259078932305025 WHERE userid = '187043';
UPDATE user_book SET isbn_0142402494 = -0.9363998747480614 WHERE userid = '187043';
UPDATE user_book SET isbn_0307444058 = 1.2249483178691107 WHERE userid = '187043

UPDATE user_book SET isbn_0140481877 = -0.6111484477037297 WHERE userid = '187043';
UPDATE user_book SET isbn_0822213419 = -0.47475268797546166 WHERE userid = '187043';
UPDATE user_book SET isbn_0451522664 = 0.3121459258414697 WHERE userid = '187043';
UPDATE user_book SET isbn_158049580X = -0.9888597823358566 WHERE userid = '187043';
UPDATE user_book SET isbn_0140482024 = -1.8806782113283784 WHERE userid = '187043';
UPDATE user_book SET isbn_185459138X = -1.849202266775702 WHERE userid = '187043';
UPDATE user_book SET isbn_0486270629 = 0.5534615007453286 WHERE userid = '187043';
UPDATE user_book SET isbn_1580495931 = 1.6131516340187966 WHERE userid = '187043';
UPDATE user_book SET isbn_1559360828 = 0.2282100737009974 WHERE userid = '187043';
UPDATE user_book SET isbn_1559360968 = 1.434787948220292 WHERE userid = '187043';
UPDATE user_book SET isbn_0451528921 = 1.2354402993866695 WHERE userid = '187043';
UPDATE user_book SET isbn_0451523377 = 0.25968601825367443 WHERE userid = '187043';

...

UPDATE user_book SET isbn_0143037234 = -0.7370522259144389 WHERE userid = '187043';
UPDATE user_book SET isbn_140003471X = 0.28066998128879267 WHERE userid = '187043';
UPDATE user_book SET isbn_0060751576 = 0.38558979646438357 WHERE userid = '187043';
UPDATE user_book SET isbn_009947915X = 0.39608177798194244 WHERE userid = '187043';
UPDATE user_book SET isbn_0142004812 = -1.2511593202748337 WHERE userid = '187043';
UPDATE user_book SET isbn_0099490625 = -1.2616513017923932 WHERE userid = '187043';
UPDATE user_book SET isbn_0143036378 = 1.4557719112554104 WHERE userid = '187043';
UPDATE user_book SET isbn_0312425023 = -1.4085390430382199 WHERE userid = '187043';
UPDATE user_book SET isbn_0061120073 = 1.0256006690354877 WHERE userid = '187043';
UPDATE user_book SET isbn_0312424442 = -2.289865490513183 WHERE userid = '187043';
UPDATE user_book SET isbn_0375806709 = 1.067568595105724 WHERE userid = '187043';
UPDATE user_book SET isbn_0312368569 = 0.11279827700784714 WHERE userid = '187043

UPDATE user_book SET isbn_1416954791 = -0.32786494672963434 WHERE userid = '187043';
UPDATE user_book SET isbn_1596912707 = -0.8209880780549111 WHERE userid = '187043';
UPDATE user_book SET isbn_0060909994 = 0.16525818459564237 WHERE userid = '187043';
UPDATE user_book SET isbn_0553377833 = -0.13900927941357114 WHERE userid = '187043';
UPDATE user_book SET isbn_030738733X = 0.5534615007453286 WHERE userid = '187043';
UPDATE user_book SET isbn_0446350567 = 0.04984638790249304 WHERE userid = '187043';
UPDATE user_book SET isbn_0345471725 = -0.9154159117129427 WHERE userid = '187043';
UPDATE user_book SET isbn_1416928073 = 0.5324775377102104 WHERE userid = '187043';
UPDATE user_book SET isbn_0394753518 = -0.8000041150197934 WHERE userid = '187043';
UPDATE user_book SET isbn_0679743111 = 0.18624214763076102 WHERE userid = '187043';
UPDATE user_book SET isbn_1931686114 = 0.0603383694200519 WHERE userid = '187043';
UPDATE user_book SET isbn_081091493X = -0.5796725031510521 WHERE userid = '18

UPDATE user_book SET isbn_0345420071 = 0.028862424867374382 WHERE userid = '187043';
UPDATE user_book SET isbn_034542008X = 0.08132233245516962 WHERE userid = '187043';
UPDATE user_book SET isbn_0345333152 = 0.039354406384933246 WHERE userid = '187043';
UPDATE user_book SET isbn_0345351452 = -0.013105501202861995 WHERE userid = '187043';
UPDATE user_book SET isbn_080713208X = 0.43804970405217886 WHERE userid = '187043';
UPDATE user_book SET isbn_0312363141 = -0.15999324244868884 WHERE userid = '187043';
UPDATE user_book SET hash_184e2bf4e10e00703b358194dfaba429 = -1.4400149875908974 WHERE userid = '187043';
UPDATE user_book SET isbn_0684853949 = -0.8629560041251475 WHERE userid = '187043';
UPDATE user_book SET isbn_0060801573 = -0.516720614045698 WHERE userid = '187043';
UPDATE user_book SET isbn_0671250671 = -2.9508603261194057 WHERE userid = '187043';
UPDATE user_book SET isbn_0679720456 = 0.21771809218343854 WHERE userid = '187043';
UPDATE user_book SET isbn_0312420609 = -0.70557628

UPDATE user_book SET isbn_030711189X = 0.595429426815565 WHERE userid = '187043';
UPDATE user_book SET isbn_0553121111 = 0.5324775377102104 WHERE userid = '187043';
UPDATE user_book SET hash_fdb303ab8d3c780c1e176c50c5e9ec25 = 0.5324775377102104 WHERE userid = '187043';
UPDATE user_book SET isbn_2070415732 = -0.789512133502234 WHERE userid = '187043';
UPDATE user_book SET isbn_0440975344 = -0.5796725031510521 WHERE userid = '187043';
UPDATE user_book SET isbn_0806511532 = -1.0518116714412116 WHERE userid = '187043';
UPDATE user_book SET isbn_0679752986 = -0.5691805216334933 WHERE userid = '187043';
UPDATE user_book SET isbn_0060892994 = -0.789512133502234 WHERE userid = '187043';
UPDATE user_book SET isbn_0441172717 = -3.12922401191791 WHERE userid = '187043';
UPDATE user_book SET isbn_0739462288 = -0.9993517638534155 WHERE userid = '187043';
UPDATE user_book SET isbn_1887424350 = 0.01837044334981552 WHERE userid = '187043';
UPDATE user_book SET isbn_1400078776 = 0.43804970405217886 WHE

OperationalError: row is too big: size 8168, maximum size 8160


In [122]:
cursor.execute("rollback")

In [124]:
i

1235

In [117]:
# userid = trial.iloc[0,0]
# sub = trial.iloc[0:5]
# index  = list(sub.isbn)
# values = list(sub.rating_delta_std)
# pairs = []
# for a in zip(index, values):
#     pair = str(a[0])+' = '+str(a[1])
#     pairs.append(pair)
# statement = 'UPDATE book_user SET'+', '.join(pairs)+" WHERE userid = '" + str(userid) +"';"
# statement

In [105]:
cursor.execute('DROP TABLE user_book;')

In [114]:
cursor.execute("UPDATE user_book SET isbn_0762447699 = 100 WHERE userid = '11372510';")

In [113]:
cursor.execute("rollback")